In [ ]:
import os, re
import pandas as pd
import altair as alt

from py_rhyme import PyRhyme


# Info about already finished
simulations = {}
prefix = './..'

for directory in os.listdir(prefix):
    if re.match('^iliev-7-128-.*-RadameshHydro$', directory):
        sname = directory.replace('-RadameshHydro', '')
        rhyme_dir = '/'.join([directory, sname])
        first_file = os.listdir('/'.join([prefix, rhyme_dir]))[0]
        
        simulations[sname] = sim = {}
        sim['id'] = sname
        sim['py_rhyme'] = PyRhyme('/'.join([prefix, rhyme_dir, first_file]))
        sim['path'] = { 'prefix': prefix, 'rhyme_dir': rhyme_dir, 'file': first_file }


# Directory structure
for d in simulations.keys():
    if not os.path.exists(d):
        os.makedirs(d)


# Ionization Front
Extracting I-Front from data for each simulation and storing them into `ifronts` object.

In [ ]:
ifronts = {}
v_conv = 3.086e+19 / 3.154e+13

for s in simulations:
    ifronts[s] = { 't': [], 'x': [], 'v': [], 'lines': {}, 'units': {
        't': 'Myr', 'x': 'kpc', 'v': 'km s^-1'
    }}
    
for k, s in simulations.items():
    if k in ifronts and ifronts[k]['t'] and ifronts[k]['x']:
        continue

    ifront = ifronts[k]
    
    r = s['py_rhyme']
    nsnaps = r.ds.num_of_snapshots()
    
    p1 = [0, .5, .5]
    p2 = [1, .5, .5]
    
    for i in range(1, 300):
        r.ds.jump_to(i)
        ifront['lines'][i] = line = r.lineout(p1, p2, 'ntr_frac_0.', n_intervals=128)
        ifront['t'].append(r.ds.time())
        ifront['x'].append(line['x'][line['y'].index(min(line['y']))] * 1000)
        if i == 1:
            ifront['v'].append(ifront['x'][-1] / ifront['t'][-1] * v_conv)
        else:
            ifront['v'].append((ifront['x'][-1] - ifront['x'][-2]) / (ifront['t'][-1] - ifront['t'][-2]) * v_conv)
         

Evolution of I-Front position

In [ ]:
sid = 0
sims = list(simulations.keys())

ifront_position = pd.DataFrame({'t': ifronts[sims[sid]]['t'], 'x': ifronts[sims[sid]]['x']})

alt.Chart(ifront_position).mark_line(color='black').encode(
    alt.X('t', scale=alt.Scale(domain=(-3, 50), clamp=True)),
    alt.Y('x', scale=alt.Scale(domain=(4, 5.8), clamp=True))
)

Evolution of I-Front velocity

In [ ]:
sid = 0
sims = list(simulations.keys())

ifront_velocity = pd.DataFrame({'t': ifronts[sims[sid]]['t'], 'v': ifronts[sims[sid]]['v']})

alt.Chart(ifront_velocity).mark_line(color='black').encode(
    alt.X('t', scale=alt.Scale(domain=(-3, 50), clamp=True)),
    alt.Y('v', scale=alt.Scale(domain=(-0.3, 3), clamp=True))
)